<a href="https://colab.research.google.com/github/secutron/RunTime/blob/master/GPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+4">G Pipeline</font></b>

<b><font color="black" size="+2">Based on:</font></b>

**GitHub repository**: [face-Image-Motion-model](http://github.com/tg-bomze/Face-Image-Motion-Model)

**GitHub repository**: [first-order-model](https://github.com/AliaksandrSiarohin/first-order-model)


---

To get started, click on the buttons (where the red arrow indicates) in each block in turn. After clicking, wait until the execution is complete.



In [1]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Clone the repository and install all requirements</font></b>

#@markdown **If there are errors in the following blocks, you can change the mirror below or check this video to use your own Google Drive: https://youtu.be/j9Yq6t4hUeA**
mirror = "Google Drive" #@param ["Gofile.io", "Google Drive"]

%cd /content
!rm -rf first_order_model
!git clone https://github.com/AliaksandrSiarohin/first-order-model first_order_model
!rm -rf sample_data
%cd first_order_model
!mkdir frames

if mirror == "Gofile.io":
  !curl -A 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0' https://srv-file22.gofile.io/getUpload?c=SUyfwc
  !wget -U "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0" https://srv-file22.gofile.io/download/SUyfwc/vox-cpk.pth.tar -O vox-cpk.pth.tar
  !wget -U "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0" https://srv-file22.gofile.io/download/SUyfwc/shape_predictor_68_face_landmarks.dat.bz2 -O shape_predictor_68_face_landmarks.dat.bz2
  !wget -U "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0" https://srv-file22.gofile.io/download/SUyfwc/align.zip -O align.zip
elif mirror == "Google Drive":
  !gdown https://drive.google.com/uc?id=1jvzD5ef6mPHRpISATDLqDMn90V-qNkV0
  !gdown https://drive.google.com/uc?id=1ONAUiyUsngDOBQFwmnqtxC2uFWlId0n2
  !gdown https://drive.google.com/uc?id=1urTYmmckJPPXVKpkriqaUDb7InUi7AP3

# provoke exception if file doesn't exist, so that it doesn't fail silently
with open('align.zip') as testopen:
  pass
!unzip -u align.zip
!pip install ffmpeg
!pip install torchvision==0.5
!pip install torch==1.4

import imageio
import numpy as np
import torch
import glob
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
from IPython.display import clear_output
import io
import base64
import warnings
import os
import cv2
from google.colab import files
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte
warnings.filterwarnings("ignore")
%matplotlib inline
#clear_output()

/content
Cloning into 'first_order_model'...
remote: Enumerating objects: 236, done.
remote: Total 236 (delta 0), reused 0 (delta 0), pack-reused 236
Receiving objects: 100% (236/236), 71.46 MiB | 45.08 MiB/s, done.
Resolving deltas: 100% (117/117), done.
/content/first_order_model
Downloading...
From: https://drive.google.com/uc?id=1jvzD5ef6mPHRpISATDLqDMn90V-qNkV0
To: /content/first_order_model/vox-cpk.pth.tar
729MB [00:03, 201MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ONAUiyUsngDOBQFwmnqtxC2uFWlId0n2
To: /content/first_order_model/shape_predictor_68_face_landmarks.dat.bz2
64.0MB [00:00, 89.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1urTYmmckJPPXVKpkriqaUDb7InUi7AP3
To: /content/first_order_model/align.zip
100% 3.32k/3.32k [00:00<00:00, 5.04MB/s]
Archive:  align.zip
  inflating: align_images.py         
  inflating: face_alignment.py       
  inflating: landmarks_detector.py   
  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6084

In [15]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upload a square video in mp4</font></b>

#@markdown **You can crop video here: https://ezgif.com/crop-video**

#@markdown **Convert video here: https://convert-video-online.com**

#@markdown ---

#@markdown *For example, you can use this available materials: https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw*
!rm -rf video
!mkdir video
uploaded = list(files.upload().keys())

if len(uploaded) > 1:
  raise ValueError('You cannot upload more than one video at a time!')

vid = uploaded[0]
os.rename(vid, vid.replace(" ", ""))
vid = vid.replace(" ", "")
!mv -f $vid video/driving.mp4
vid = 'video/driving.mp4'

fps_of_video = int(cv2.VideoCapture(vid).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(vid).get(cv2.CAP_PROP_FRAME_COUNT))

#clear_output()

Saving b2-scale512.mp4 to b2-scale512.mp4


In [4]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upload one or more face photos</font></b>

!rm -rf raw_images aligned_images
!mkdir raw_images aligned_images
uploaded = files.upload()

i = 0
raw_photolist = []
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  pho = "photo-" + str(i) + "." + fn.split(".")[1]
  !mv -f $fn raw_images/$pho
  raw_photolist.append(pho)
  i += 1

#clear_output()

Saving koo512-00001.png to koo512-00001.png
User uploaded file "koo512-00001.png" with length 177137 bytes


In [5]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Crop face in photos</font></b>

success = False
try:
  !python align_images.py raw_images/ aligned_images/
  pho_file = !ls aligned_images
  if len(pho_file) == 0:
    !cp -r raw_images/ aligned_images/
  else:
    success = True
except BaseException:
  !cp -r raw_images/ aligned_images/

aligned_photolist = []
for photoname in raw_photolist:
  newname = photoname.split('.')[0] + '_01.png'
  aligned_photolist.append(newname if success else photoname)

#clear_output()

Using TensorFlow backend.
2020-07-27 10:24:08.001531: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Aligning photo-0.png ...
Getting landmarks...
Starting face alignment...
Wrote result aligned_images/photo-0_01.png


In [16]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Transform image</font></b>

#@markdown *Attention! The image is transformed only visually. Sound from the video is not transferred. In addition, FPS decreases, so to restore it, then execute the next block.*
!rm -rf video/intermediate video/final
!mkdir -p video/intermediate video/final

print('Preparing videos and photos for transformation')
source_images = []
for photoname in aligned_photolist:
  source_image = imageio.imread('aligned_images/' + photoname)
  source_image = resize(source_image, (256, 256))[..., :3]
  source_images.append(source_image)

placeholder_bytes = base64.b64decode('iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mP8/x8AAwMCAO+ip1sAAAAASUVORK5CYII=')
placeholder_image = imageio.imread(placeholder_bytes, '.png')
placeholder_image = resize(placeholder_image, (256, 256))[..., :3]

print('*** 1')

#driving_video = imageio.mimread(vid, memtest=False)

reader = imageio.get_reader(vid)
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass


print('*** 1-2')
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

print('*** 2')

def display(source, driving, generated=None):
  fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
  ims = []
  for i in range(len(driving)):
    cols = [[placeholder_image], []]
    for sourceitem in source:
      cols[0].append(sourceitem)
    cols[1].append(driving[i])
    if generated is not None:
      for generateditem in generated:
        cols[1].append(generateditem[i])

    endcols = []
    for thiscol in cols:
      endcols.append(np.concatenate(thiscol, axis=1))
    
    im = plt.imshow(np.vstack(endcols), animated=True) # np.concatenate(cols[0], axis=1)
    plt.axis('off')
    ims.append([im])
  ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
  plt.close()
  return ani

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                                          checkpoint_path='vox-cpk.pth.tar')

#clear_output()
print('Start the transformation')
videolist = []
predictionlist = []
i = 0
for source_img in source_images:
  videoname = 'result-' + str(i) + '.mp4'
  print('Generating ' + videoname)
  predictions = make_animation(source_img, driving_video, generator, kp_detector, relative=True)
  imageio.mimsave('video/intermediate/' + videoname, [img_as_ubyte(frame) for frame in predictions])
  videolist.append(videoname)
  predictionlist.append(predictions)
  i += 1

#clear_output()
print('Videos generated. Wait a few seconds...')
HTML(display(source_images, driving_video, predictionlist).to_html5_video())

Preparing videos and photos for transformation
*** 1
*** 1-2
*** 2
Start the transformation
Generating result-0.mp4


100%|██████████| 667/667 [00:23<00:00, 28.42it/s]


Videos generated. Wait a few seconds...


In [17]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> FPS Recovery</font></b>
fps_of_video = int(cv2.VideoCapture(vid).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(vid).get(cv2.CAP_PROP_FRAME_COUNT))

!rm -rf frames
!mkdir frames

play_video = True #@param {type:"boolean"}
play_html = ''

add_audio = True #@param {type:"boolean"}
if add_audio == True:
  !ffmpeg -y -i $vid -vn -ar 44100 -ac 2 -ab 192K -f mp3 sound.mp3

for videoname in videolist:
  vidcap = cv2.VideoCapture('video/intermediate/' + videoname)
  success,image = vidcap.read()
  count = 0
  success = True
  while success:
    cv2.imwrite("frames/frame%09d.jpg" % count, image)
    success,image = vidcap.read()
    count += 1

  frames = []
  img = os.listdir("frames/")
  img.sort()
  for i in img:
    frames.append(imageio.imread("frames/"+i))
  frames = np.array(frames)
  dstvid = 'video/final/' + videoname
  imageio.mimsave(dstvid, frames, fps=fps_of_video)

  !rm -rf frames
  !mkdir frames
  
  print('Assembly completed for ' + videoname)
  
  if add_audio == True:
    tmpfile = dstvid.replace('.mp4', '-audio.mp4')
    !ffmpeg -i sound.mp3 -i $dstvid $tmpfile
    !rm -rf $dstvid
    !mv -f $tmpfile $dstvid

  if play_video == True:
    video = io.open(dstvid, 'r+b').read()
    encoded = base64.b64encode(video)
    play_html = play_html + ('<video alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /> </video>'.format(encoded.decode('ascii')))
  
#clear_output()
HTML(data=play_html)

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download final videos</font></b>
lsinfo = !ls video/final
if lsinfo.count('.mp4') == 1:
  files.download('video/final/result-0.mp4')
else:
  !zip -r ../export.zip video/final
  files.download('../export.zip')

updating: video/final/ (stored 0%)
updating: video/final/result-0.mp4 (deflated 6%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


---
<b><font color="red" size="+3">Additional modules:</font></b>

***Next come optional blocks from my other project [ENTAR](https://github.com/tg-bomze/ENTAR)***

# <b><font color="gree" size="+2">Frames interpolation ([VFIASC](https://github.com/sniklaus/sepconv-slomo))

*↓ Click to open section ↓*

Creator: [Simon Niklaus](http://sniklaus.com/about/welcome)

In [ ]:
#@title ##**Clone the repository and download the necessary components** { display-mode: "form" }
%cd /content/
!git clone https://github.com/sniklaus/sepconv-slomo.git VFIASC
%cd /content/VFIASC
!rm -rf network-l1.pytorch
!rm -rf network-lf.pytorch
try:
  !gdown https://drive.google.com/uc?id=1v77wNU8sYh0hBmljeGt5LzY9ehQU0elv
  !gdown https://drive.google.com/uc?id=1na11Ey0TB1KEDps4uEwyTBsDcmpo2kr6
except BaseException:
  !bash download.bash
clear_output()

In [ ]:
#@title ##**Install the necessary dependencies** { display-mode: "form" }
#@markdown **This block is required. Execution time: ~ 12 minutes**
print('You can go for a coffee while.\n')
!pip install cupy
!pip install moviepy
clear_output()
print("It's time to do the next block.")

In [ ]:
#@title ##**Interpolate frames** { display-mode: "form" }
#@markdown **How many times increase FPS:**
fps_boosting = 'x4' #@param ["x4", "x16"]
%env CUDA_VISIBLE_DEVICES=0
if fps_boosting == 'x4':
  !python run.py --model lf --video /content/final.mp4 --out ./video_x4.mp4
  video_name = "video_x4.mp4"
elif fps_boosting == 'x16':
  !python run.py --model lf --video /content/final.mp4 --out ./video_x4.mp4
  !python run.py --model lf --video ./video_x4.mp4 --out ./video_x16.mp4
  video_name = "video_x16.mp4"
clear_output()

mfps_of_video = int(cv2.VideoCapture(video_name).get(cv2.CAP_PROP_FPS))
mframes_of_video = int(cv2.VideoCapture(video_name).get(cv2.CAP_PROP_FRAME_COUNT))
#@markdown *I recommend installing* **x4**. *Otherwise, the execution of this block may be delayed or lead to an error. On top of that, the effect of SlowMo will be unnatural*

In [ ]:
#@title ##**Convert SlowMo video to high-frequency** { display-mode: "form" }
!rm -rf output.mp4
!rm -rf slowed_movie_frames
!rm -rf /content/final.mp4
!rm -rf /content/fps_boosted_video.mp4

fr = int(fps_of_video*int(fps_boosting.split("x")[1]))
!mkdir 'slowed_movie_frames'
vidcap = cv2.VideoCapture(video_name)
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("slowed_movie_frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

staffs = []
img = os.listdir("slowed_movie_frames/")
img.sort()
clear_output()
for i in img:
  staffs.append("slowed_movie_frames/"+i)

staff = cv2.imread(staffs[0])  # get size from the 1st frame
writer = cv2.VideoWriter(
    'output.mp4',
    cv2.VideoWriter_fourcc(*'MP4V'),   # codec (*'DIVX', *'MP4V', *'FMP4', *'MJPG', *'XVID', *'MP4S')
    fr,  # fps
    (staff.shape[1], staff.shape[0]),  # width, height
    isColor=len(staff.shape) > 2)
for staff in map(cv2.imread, staffs):
    writer.write(staff)
writer.release()
clear_output()

In [ ]:
#@title ##**Download the result** { display-mode: "form" }
!cp -r output.mp4 /content/fps_boosted_video.mp4
!cp -r output.mp4 /content/final.mp4
files.download("/content/fps_boosted_video.mp4")

---
# <b><font color="gree" size="+2">Upscale Resolution ([ESRGAN](https://github.com/xinntao/ESRGAN))

*↓ Click to open section ↓*

Creator: [Xintao Wang](https://xinntao.github.io)

In [ ]:
#@title ##**Clone the repository and download the necessary components** { display-mode: "form" }
%cd /content/
!git clone https://github.com/xinntao/ESRGAN.git
!cp -r /content/final.mp4 ESRGAN/
%cd /content/ESRGAN
!git checkout tags/old-arch
model_url = "https://www.dropbox.com/s/vouc15j8jjp2o5n/RRDB_ESRGAN_x4_old_arch.pth?dl=0"
!wget $model_url --content-disposition -P models
import architecture as arch
import os.path
!mkdir frames
!rm -rf results/baboon_ESRGAN.png
clear_output()

In [ ]:
!pwd
#@title ##**Split the video into frames** { display-mode: "form" }
frames_of_video = int(cv2.VideoCapture("final.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
fps_of_video = int(cv2.VideoCapture("final.mp4").get(cv2.CAP_PROP_FPS))
vidcap = cv2.VideoCapture('final.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

/content/ESRGAN


In [ ]:
#@title ##**Upscale resolution and improve frame quality** { display-mode: "form" }
#@markdown **How many times to upscale the resolution:**
upscale = 4 #@param {type: "slider", min: 4, max: 8}
%env CUDA_VISIBLE_DEVICES=0
device = torch.device('cuda')
model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=upscale, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
model.load_state_dict(torch.load('models/{:s}'.format('RRDB_ESRGAN_x4_old_arch.pth')), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)

count_frames = 0

for path in glob.glob('frames/*'):
    base = os.path.splitext(os.path.basename(path))[0]
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    path = 'results/{:s}_rlt.png'.format(base)
    cv2.imwrite(path, output)
    count_frames += 1
    clear_output()
    print("Processed: {} из {}".format(str(count_frames), str(frames_of_video)))
clear_output()

In [ ]:
#@title ##**Collecting frames in a video file** { display-mode: "form" }
frames = []
img = os.listdir("results/")
img.sort()
for i in img:
  frames.append(imageio.imread("results/"+i))
frames = np.array(frames)
imageio.mimsave("upscale_video.mp4", frames, fps=fps_of_video)

print('Сборка завершена')
!cp -r upscale_video.mp4 /content/upscale_video.mp4
!cp -r upscale_video.mp4 /content/final.mp4
clear_output()

video = io.open('/content/upscale_video.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" width="400" height="400" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /> </video>'''.format(encoded.decode('ascii')))

In [ ]:
#@title ##**Download the result** { display-mode: "form" }
!cp -r upscale_video.mp4 /content/upscale_video.mp4
!cp -r upscale_video.mp4 /content/final.mp4
files.download("/content/upscale_video.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
# <b><font color="gree" size="+2">Enhancing video ([EDVR](https://github.com/xinntao/EDVR))
*↓ Click to open section ↓*

Creator: [Xintao Wang](https://xinntao.github.io)

In [ ]:
#@title ##**Clone the repository and upload the pre-trained model** { display-mode: "form" }
%cd /content
!pip install numpy opencv-python lmdb pyyaml
!pip install tb-nightly future
!git clone https://github.com/xinntao/EDVR
%cd /content/EDVR/codes/models/archs/dcn
!python setup.py develop
%cd /content/EDVR/codes
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1SGVehpZt4WL_X8Jh6blyqmHpc8DdImgv' -O /content/EDVR/experiments/pretrained_models/EDVR_REDS_deblurcomp_L.pth
#clear_output()

/content
Cloning into 'EDVR'...
remote: Enumerating objects: 556, done.
remote: Total 556 (delta 0), reused 0 (delta 0), pack-reused 556
Receiving objects: 100% (556/556), 471.72 KiB | 535.00 KiB/s, done.
Resolving deltas: 100% (283/283), done.
/content/EDVR/codes/models/archs/dcn
running develop
running egg_info
creating deform_conv.egg-info
writing deform_conv.egg-info/PKG-INFO
writing dependency_links to deform_conv.egg-info/dependency_links.txt
writing top-level names to deform_conv.egg-info/top_level.txt
writing manifest file 'deform_conv.egg-info/SOURCES.txt'
writing manifest file 'deform_conv.egg-info/SOURCES.txt'
running build_ext
building 'deform_conv_cuda' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/src
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/torch/include -I/usr/l

In [ ]:
#@title ##**Initialize the necessary functions** { display-mode: "form" }
import utils.util as util
import data.util as data_util
import models.archs.EDVR_arch as EDVR_arch

workfolder = Path('/content')
source_folder = workfolder / "source"
inframes_root = workfolder / "inframes"
audio_root = workfolder / "audio"
outframes_root = workfolder / "outframes"
result_folder = workfolder / "result"
pretrained_models = Path('../experiments/pretrained_models')

def clean_mem():
    # torch.cuda.empty_cache()
    gc.collect()

def get_fps(source_path: Path) -> str:
    print(source_path)
    probe = ffmpeg.probe(str(source_path))
    stream_data = next(
        (stream for stream in probe['streams'] if stream['codec_type'] == 'video'),
        None,
    )
    return stream_data['avg_frame_rate']

def preProcess(imag_path_l, multiple):
  '''Need to resize images for blurred model (needs to be multiples of 16)'''
  for img_path in imag_path_l:
    im = Image.open(img_path)
    h, w = im.size
    # resize so they are multiples of 4 or 16 (for blurred)
    h = h - h % multiple
    w = w - w % multiple
    im = im.resize((h,w))
    im.save(img_path)

def purge_images(dir):
  for f in os.listdir(dir):
    if re.search('.*?\.jpg', f):
      os.remove(os.path.join(dir, f))

def extract_raw_frames(source_path: Path):
    inframes_folder = inframes_root / (source_path.stem)
    inframe_path_template = str(inframes_folder / '%5d.jpg')
    inframes_folder.mkdir(parents=True, exist_ok=True)
    purge_images(inframes_folder)
    ffmpeg.input(str(source_path)).output(
        str(inframe_path_template), format='image2', vcodec='mjpeg', qscale=0
    ).run(capture_stdout=True)

def make_subfolders(img_path_l, chunk_size):
  i = 0
  subFolderList = []
  source_img_path = Path('/content/EDVR/codes/video/inframes/video_subfolders')
  source_img_path.mkdir(parents=True, exist_ok=True)
  for img in img_path_l:
    if i % chunk_size == 0:
      img_path = source_img_path / str(i)
      img_path.mkdir(parents=True, exist_ok=True)
      subFolderList.append(str(img_path))
    i+=1
    img_name = osp.basename(img)
    img_path_name = img_path / img_name
    shutil.copyfile(img, img_path_name)

  return subFolderList

def remove_subfolders():
  shutil.rmtree('/content/EDVR/codes/video/inframes/video_subfolders', ignore_errors=True, onerror=None)

def edvrPredict(data_mode, chunk_size, stage):
  device = torch.device('cuda')
  os.environ['CUDA_VISIBLE_DEVICES'] = '0'
  data_mode = "blur_comp"
  stage = stage  # 1 or 2, use two stage strategy for REDS dataset.
  flip_test = False
  model_path = pretrained_models / 'EDVR_REDS_deblurcomp_L.pth'   

  print('Model Used: ', model_path)
  
  if data_mode == 'Vid4':
      N_in = 7  # use N_in images to restore one HR image
  else:
      N_in = 5

  predeblur, HR_in = False, False
  back_RBs = 40
  if data_mode == 'blur' or data_mode == 'blur_comp':
      predeblur, HR_in = True, True
  if stage == 2:
      HR_in = True
      back_RBs = 20
  if data_mode == 'TOF':
    model = TOF_arch.TOFlow(adapt_official=True)
  else:
    model = EDVR_arch.EDVR(128, N_in, 8, 5, back_RBs, predeblur=predeblur, HR_in=HR_in)

  #### dataset
  test_dataset_folder = '/content/EDVR/codes/video/inframes'

  #### evaluation
  crop_border = 0
  border_frame = N_in // 2  # border frames when evaluate
  # temporal padding mode
  if data_mode in ('Vid4','sharp_bicubic'):
      padding = 'new_info'
  else:
      padding = 'replicate'
  save_imgs = True

  save_folder = '/content/EDVR/codes/video/outframes'
  util.mkdirs(save_folder)

  #### set up the models
  model.load_state_dict(torch.load(model_path), strict=True)
  model.eval()
  model = model.to(device)

  avg_psnr_l, avg_psnr_center_l, avg_psnr_border_l = [], [], []
  subfolder_name_l = []
  # remove old video_subfolder if exists
  remove_subfolders()
  subfolder_l = sorted(glob.glob(osp.join(test_dataset_folder, '*')))

  # for each subfolder
  for subfolder in subfolder_l:
      subfolder_name = osp.basename(subfolder)
      subfolder_name_l.append(subfolder_name)
      save_subfolder = osp.join(save_folder, subfolder_name)

      img_path_l = sorted(glob.glob(osp.join(subfolder, '*')))
      if save_imgs:
          util.mkdirs(save_subfolder)
          purge_images(save_subfolder)

      # preprocess images (needed for blurred models)
      if predeblur:
        preProcess(img_path_l, 16)
      else:
        preProcess(img_path_l, 4)
      # make even more subfolders
      subFolderList = make_subfolders(img_path_l, chunk_size)

      #### read LQ and GT images in chunks of 1000
      for subSubFolder in subFolderList:
        clean_mem()
        imgs_LQ = data_util.read_img_seq(subSubFolder)
        subSubFolder_l = sorted(glob.glob(osp.join(subSubFolder, '*')))
        max_idx = len(subSubFolder_l)
        avg_psnr, avg_psnr_border, avg_psnr_center, N_border, N_center = 0, 0, 0, 0, 0

        # process each image
        for img_idx, img_path in tqdm(enumerate(subSubFolder_l)):
            img_name = osp.splitext(osp.basename(img_path))[0]
            select_idx = data_util.index_generation(img_idx, max_idx, N_in, padding=padding)
            imgs_in = imgs_LQ.index_select(0, torch.LongTensor(select_idx)).unsqueeze(0).to(device)

            if flip_test:
                output = util.flipx4_forward(model, imgs_in)
            else:
                output = util.single_forward(model, imgs_in)
            output = util.tensor2img(output.squeeze(0))

            if save_imgs:
                cv2.imwrite(osp.join(save_subfolder, '{}.jpg'.format(img_name)), output)
                # print('Saved Image:', str(osp.join(save_subfolder, '{}.jpg'.format(img_name))))

def moveProcessedFrames():
  shutil.rmtree('/content/EDVR/codes/video/inframes')
  os.rename('/content/EDVR/codes/video/outframes', '/content/EDVR/codes/video/inframes')

def build_video(source_path: Path) -> Path:
        out_path = result_folder / (
            source_path.name.replace('.mp4', '_no_audio.mp4')
        )
        outframes_folder = outframes_root / (source_path.stem)
        outframes_path_template = str(outframes_folder / '%5d.jpg')
        out_path.parent.mkdir(parents=True, exist_ok=True)
        if out_path.exists():
            out_path.unlink()
        fps = get_fps(source_path)
        print('Original FPS is: ', fps)

        ffmpeg.input(
            str(outframes_path_template),
            format='image2',
            vcodec='mjpeg',
            framerate=fps,
        ).output(str(out_path), crf=17, vcodec='libx264').run(capture_stdout=True)

        result_path = result_folder / source_path.name
        if result_path.exists():
            result_path.unlink()
        # making copy of non-audio version in case adding back audio doesn't apply or fails.
        shutil.copyfile(str(out_path), str(result_path))

        # adding back sound here
        audio_file = Path(str(source_path).replace('.mp4', '.aac'))
        if audio_file.exists():
            audio_file.unlink()

        os.system(
            'ffmpeg -y -i "'
            + str(source_path)
            + '" -vn -acodec copy "'
            + str(audio_file)
            + '"'
        )

        if audio_file.exists:
            os.system(
                'ffmpeg -y -i "'
                + str(out_path)
                + '" -i "'
                + str(audio_file)
                + '" -shortest -c:v copy -c:a aac -b:a 256k "'
                + str(result_path)
                + '"'
            )
        print('Video created here: ' + str(result_path))
        return result_path

def edvr_video(source_path: Path, chunk_size: int):
    # extract frames
    extract_raw_frames(source_path)

    # process frames
    edvrPredict("blur_comp", chunk_size, 1)

    # build back video
    build_video(source_path)

NameError: ignored

In [ ]:
#@title ##**Enhancing the quality of frames** { display-mode: "form" }
%%time
edvr_video(Path('/content/final.mp4'), 100)
clear_output()

In [ ]:
#@title ##**Get result** { display-mode: "form" }
!rm -rf /content/video.mp4
!cp -r /content/EDVR/codes/video/result/final.mp4 /content/video.mp4
!cp -r /content/EDVR/codes/video/result/final.mp4 /content/enhanced_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/enhanced_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('/content/enhanced_video.mp4')

In [ ]:
#@title ##**Exit EDVR** { display-mode: "form" }
#@markdown *If you are going to continue to carry out the following points, then first of all, be sure to complete this block!*
!rm -rf /content/final.aac
!cp -r /content/EDVR/codes/video/result/final.mp4 /content/video.mp4
!cp -r /content/EDVR/codes/video/result/final.mp4 /content/enhanced_video.mp4
%cd /content
clear_output()